In [1]:
# Parameters
image_path = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\data\\test\\images\\test_pid.png"
outputs_dir = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\outputs"
data_dir = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\data\\dummy"


In [2]:
import json
import pandas as pd
from pathlib import Path

In [3]:
outputs_dir = Path(outputs_dir)
data_dir = Path(data_dir)

readings_path   = data_dir / "values.json"
thresholds_path = data_dir / "threshold.json"
anomalies_path  = outputs_dir / "anomalies.json"

with open(readings_path, encoding="utf-8") as f:
    readings = json.load(f)

with open(thresholds_path, "r", encoding="utf-8") as f:
    thresholds = json.load(f)


In [4]:
readings_df = pd.json_normalize(readings["pressure_readings"])

readings_df["timestamp"] = pd.to_datetime(readings_df["timestamp"])

thr_records = []
for k, v in thresholds["pressure_thresholds"].items():
    row = {"component": k, **v}
    thr_records.append(row)
thresholds_df = pd.DataFrame(thr_records)

alerts = [] 

# map thresholds
thr_map = {
    row["component"]: {"min": row["min"], "max": row["max"], "units": row["units"]}
    for _, row in thresholds_df.iterrows()
}

components = [c for c in readings_df.columns if c != "timestamp"]

for _, row in readings_df.sort_values("timestamp").iterrows():
    ts = row["timestamp"].isoformat()
    for comp in components:
        val = float(row[comp])
        min_v = thr_map[comp]["min"]
        max_v = thr_map[comp]["max"]
        units = thr_map[comp]["units"]
        if not (min_v <= val <= max_v):
            alerts.append({
                "component": comp,
                "observed_value": val,
                "timestamp": ts,
                "expected_range": f"{min_v} - {max_v} {units}"
            })

alerts_df = pd.DataFrame(alerts)

with open(anomalies_path, "w", encoding="utf-8") as f:
    json.dump(alerts, f, indent=2, ensure_ascii=False)